In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder 
from recbole.config import Config
from recbole.data import create_dataset, data_preparation

from logging import getLogger
from recbole.model.general_recommender import BPR, Pop
from recbole.model.sequential_recommender import GRU4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

import torch
from recbole.data.interaction import Interaction

import make_atomic_files as at

## Tuning
from recbole.trainer import HyperTuning
from recbole.utils import init_seed
import os
from recbole.utils import get_model, get_trainer

In [13]:
at.og_atomic_files()

## Tuning

In [3]:
def objective_function(config_dict=None, config_file_list=None):
    k = 10

    config_dict = {
        'model': 'GRU4Rec',
        'train_neg_sample_args': None,
        'seed': 1234,
        'reproducibility': False,
        'topk': 10,
        'use_gpu': True
    } 

    config = Config(config_dict=config_dict, config_file_list=['foursquare_general.yaml'])
    init_seed(config['seed'], config['reproducibility'])
    dataset = create_dataset(config)
    train_data, valid_data, test_data = data_preparation(config, dataset)
    model_name = config['model']
    model = get_model(model_name)(config, train_data._dataset).to(config['device'])
    trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)
    test_result = trainer.evaluate(test_data)

    return {
        'model': model_name,
        'best_valid_score': best_valid_score,
        'valid_score_bigger': config['valid_metric_bigger'],
        'best_valid_result': best_valid_result,
        'test_result': test_result
    }

hp = HyperTuning(objective_function=objective_function, algo='random', early_stop=10, 
                max_evals=10, params_file='caser.hyper', fixed_config_file_list=['foursquare_general.yaml'])


In [4]:
def get_history(inter):

    visits = []
    set_uid = set(inter['uid:token'])
    for u in set_uid:
        visits.append(inter[inter['uid:token'] == u]['venue_id:token'].values.tolist())
    
    return np.array(visits)

inter = pd.read_csv('foursquare/foursquare.inter', sep='\t')

hp.run()
params = hp.best_params
print(params)

running parameters:                                   
{'dropout_prob': 0.0, 'hidden_size': 128, 'learning_rate': 0.01, 'num_layers': 1}
  0%|          | 0/10 [00:21<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

## Training

In [17]:
## Train General Recommendation
#params = {'dynamic': False, 'embedding_size': 128, 'learning_rate': 0.07885866707608949, 'mlp_hidden_size': '[64,64,64]', 'sample_num': 5, 'train_batch_size': 4096}
# configurations initialization
k = 10

params = {
    'model': 'GRU4Rec',
    'train_neg_sample_args': None,
    'seed': 1234,
    'reproducibility': False,
    'topk': 10,
    'use_gpu': True,
} 

config = Config(model='GRU4Rec', dataset='foursquare', config_file_list=['foursquare_general.yaml'], config_dict = params)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# dataset creating and filtering
dataset = create_dataset(config)

# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

model = GRU4Rec(config, train_data.dataset).to(config['device'])

# trainer loading and initialization
trainer = Trainer(config, model)

In [ ]:

# model training
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)

# model evaluation
test_result = trainer.evaluate(test_data)
print(test_result)

## Make predictions

In [ ]:
## Make prediction
visits = get_history(inter)

unique_users = list(set(inter['uid:token']))
unique_locations = list(set(inter['venue_id:token']))

current_time = max(inter['timestamp:token'])+1

#make prediction for users
input_inter = Interaction({
    'uid': torch.tensor(unique_users),
    'venue_id': torch.tensor(visits)
})

with torch.no_grad():
    scores = model.full_sort_predict(input_inter).reshape((len(unique_users), -1))

#length |items| + 1 because of the padding
print(scores.shape)

# get the 10 items with highest scores
rec_list = np.argsort(scores, axis = 1)[:, -k:]

# select one item at random for each user
def random_choice(a):
    # select one item, but then translated back
    r_c = np.random.choice(a, 1)
    return int(r_c) - 1

random_item = np.apply_along_axis(random_choice, 1, rec_list)
print(current_time)

In [ ]:
## Add new values in the dataset
new_locations = pd.DataFrame({'uid:token': unique_users, 'venue_id:token':random_item.tolist(), 'timestamp:token':[current_time]*len(random_item)}, columns=['uid:token', 'venue_id:token', 'timestamp:token'])
new_locations.head()

inter = pd.concat([inter, new_locations], axis = 0).reset_index(drop = True)
inter.sort_values(by=['uid:token', 'timestamp:token'], inplace=True)

unique_users = list(set(inter['uid:token']))
unique_locations = list(set(inter['venue_id:token']))

inter.to_csv('foursquare/foursquare.inter', index=False, sep = '\t')